In [ ]:
# 01. Data Quality Analysis and Cleaning (Fixed)
"""
This script is a fixed version of the data quality notebook, with robust handling for missing values visualization.
"""

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os

In [ ]:
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
np.random.seed(42)

In [ ]:
# Load raw data
df = pd.read_csv('../Synthetic_ESG_Greenwashing_Dataset_200_v2.csv')
print(f"Dataset shape: {df.shape}")
display(df.head())

In [ ]:
# 1. Data Schema Validation
print("Data Types:")
print(df.dtypes)
print("\nDataset Info:")
print(df.info())

In [ ]:
expected_columns = [
    'project_id', 'organization_name', 'report_year', 'esg_claim_text',
    'claim_category', 'claimed_metric_type', 'claimed_value', 'measurement_unit',
    'project_location', 'actual_measured_value', 'value_deviation',
    'external_validation_score', 'greenwashing_flag', 'controversy_flag',
    'source_doc_link', 'report_sentiment_score', 'llm_claim_consistency_score', 'timestamp'
]

In [ ]:
missing_columns = set(expected_columns) - set(df.columns)
extra_columns = set(df.columns) - set(expected_columns)
print(f"\nMissing columns: {missing_columns}")
print(f"Extra columns: {extra_columns}")
print(f"All expected columns present: {len(missing_columns) == 0}")

In [ ]:
# 2. Missing Values Analysis
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing_data,
    'Missing Percent': missing_percent
})

In [ ]:
print("Missing Values Analysis:")
missing_columns = missing_df[missing_df['Missing Count'] > 0]
print(missing_columns)

In [ ]:
# Visualize missing values only if there are any
if len(missing_columns) > 0:
    plt.figure(figsize=(12, 6))
    missing_columns['Missing Percent'].plot(kind='bar')
    plt.title('Missing Values by Column')
    plt.xlabel('Columns')
    plt.ylabel('Missing Percentage')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('../reports/figures/missing_values.png', dpi=300, bbox_inches='tight')
    plt.show()
else:
    print("\n✅ No missing values found in the dataset!")
    print("This is excellent data quality - no missing value visualization needed.")
    # Optionally, show completeness
    plt.figure(figsize=(12, 6))
    completeness = (1 - missing_percent/100) * 100
    completeness.plot(kind='bar', color='green', alpha=0.7)
    plt.title('Data Completeness by Column')
    plt.xlabel('Columns')
    plt.ylabel('Completeness (%)')
    plt.xticks(rotation=45)
    plt.ylim(0, 100)
    plt.tight_layout()
    plt.savefig('../reports/figures/data_completeness.png', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
# 3. Duplicate Detection
duplicates = df.duplicated().sum()
key_columns = ['project_id', 'esg_claim_text']
duplicates_key = df.duplicated(subset=key_columns).sum()
print(f"Total duplicate rows: {duplicates}")
print(f"Duplicate rows based on {key_columns}: {duplicates_key}")
if duplicates > 0:
    print("\nDuplicate rows:")
    print(df[df.duplicated()])
else:
    print("\n✅ No duplicate rows found.")

In [ ]:
# 4. Text Quality Analysis
df['text_length'] = df['esg_claim_text'].str.len()
df['word_count'] = df['esg_claim_text'].str.split().str.len()
print("Text Length Statistics:")
print(df[['text_length', 'word_count']].describe())
fig, axes = plt.subplots(1, 2, figsize=(15, 5))
axes[0].hist(df['text_length'], bins=30, alpha=0.7)
axes[0].set_title('Distribution of Text Length (Characters)')
axes[0].set_xlabel('Character Count')
axes[0].set_ylabel('Frequency')
axes[1].hist(df['word_count'], bins=20, alpha=0.7, color='orange')
axes[1].set_title('Distribution of Word Count')
axes[1].set_xlabel('Word Count')
axes[1].set_ylabel('Frequency')
plt.tight_layout()
plt.savefig('../reports/figures/text_length_distribution.png', dpi=300, bbox_inches='tight')
plt.show()
short_texts = df[df['text_length'] < 50]
long_texts = df[df['text_length'] > 500]
print(f"\nTexts with < 50 characters: {len(short_texts)}")
print(f"Texts with > 500 characters: {len(long_texts)}")
if len(short_texts) > 0:
    print("\nSample short texts:")
    print(short_texts['esg_claim_text'].head())
else:
    print("\n✅ No very short texts found.")

In [ ]:
# 5. Data Cleaning
df_clean = df.copy()
df_clean = df_clean.drop_duplicates()
print(f"Removed {len(df) - len(df_clean)} duplicate rows")
df_clean['esg_claim_text'] = df_clean['esg_claim_text'].str.strip()
df_clean['esg_claim_text'] = df_clean['esg_claim_text'].fillna('No claim text provided')
df_clean = df_clean[df_clean['esg_claim_text'].str.len() >= 10]
print(f"Removed {len(df) - len(df_clean)} rows with very short texts")
remaining_missing = df_clean.isnull().sum().sum()
print(f"\nRemaining missing values: {remaining_missing}")

In [ ]:
# 6. Feature Engineering
import sys
sys.path.append('../src')
from data_prep import engineer_features
df_features = engineer_features(df_clean)
print(f"Original features: {len(df_clean.columns)}")
print(f"Engineered features: {len(df_features.columns)}")
print(f"Feature engineering added: {len(df_features.columns) - len(df_clean.columns)} new features")
original_cols = set(df_clean.columns)
new_cols = set(df_features.columns) - original_cols
print(f"\nNew engineered features: {list(new_cols)}")
missing_in_features = df_features.isnull().sum()
if missing_in_features.sum() > 0:
    print("\nMissing values in engineered features:")
    print(missing_in_features[missing_in_features > 0])
else:
    print("\n✅ No missing values in engineered features")

In [ ]:
# 7. Data Quality Metrics
quality_metrics = {
    'original_rows': int(len(df)),
    'cleaned_rows': int(len(df_clean)),
    'rows_removed': int(len(df) - len(df_clean)),
    'duplicates_removed': int(len(df) - len(df.drop_duplicates())),
    'missing_values_original': int(df.isnull().sum().sum()),
    'missing_values_cleaned': int(df_clean.isnull().sum().sum()),
    'text_length_min': int(df_clean['esg_claim_text'].str.len().min()),
    'text_length_max': int(df_clean['esg_claim_text'].str.len().max()),
    'text_length_mean': float(df_clean['esg_claim_text'].str.len().mean()),
    'unique_organizations': int(df_clean['organization_name'].nunique()),
    'unique_locations': int(df_clean['project_location'].nunique()),
    'category_distribution': df_clean['claim_category'].value_counts().to_dict(),
    'greenwashing_rate': float(df_clean['greenwashing_flag'].mean())
}
import json
with open('../reports/data_quality_metrics.json', 'w') as f:
    json.dump(quality_metrics, f, indent=2)
print("\nQuality metrics saved to ../reports/data_quality_metrics.json")

In [ ]:
# 8. Save Cleaned Data
os.makedirs('../data', exist_ok=True)
df_clean.to_parquet('../data/clean_claims.parquet', index=False)
df_features.attrs = {}  # Remove non-serializable attrs
df_features.to_parquet('../data/clean_claims_features.parquet', index=False)
print("Cleaned data saved:")
print("- ../data/clean_claims.parquet (cleaned original data)")
print("- ../data/clean_claims_features.parquet (with engineered features)")
print(f"\nFinal cleaned dataset shape: {df_clean.shape}")
print(f"Final features dataset shape: {df_features.shape}")
print("\nSample of cleaned data:")
print(df_clean.head())

In [ ]:
# 9. Summary
print("=== DATA QUALITY ANALYSIS SUMMARY ===\n")
print("1. DATA CLEANING RESULTS:")
print(f"   - Original rows: {quality_metrics['original_rows']}")
print(f"   - Cleaned rows: {quality_metrics['cleaned_rows']}")
print(f"   - Rows removed: {quality_metrics['rows_removed']}")
print(f"   - Duplicates removed: {quality_metrics['duplicates_removed']}")
print("\n2. MISSING VALUES:")
print(f"   - Original missing values: {quality_metrics['missing_values_original']}")
print(f"   - Remaining missing values: {quality_metrics['missing_values_cleaned']}")
print("\n3. TEXT QUALITY:")
print(f"   - Text length range: {quality_metrics['text_length_min']} - {quality_metrics['text_length_max']} characters")
print(f"   - Average text length: {quality_metrics['text_length_mean']:.1f} characters")
print("\n4. DATASET CHARACTERISTICS:")
print(f"   - Unique organizations: {quality_metrics['unique_organizations']}")
print(f"   - Unique locations: {quality_metrics['unique_locations']}")
print(f"   - Greenwashing rate: {quality_metrics['greenwashing_rate']:.2%}")
print("\n5. FEATURE ENGINEERING:")
print(f"   - Original features: {len(df_clean.columns)}")
print(f"   - Engineered features: {len(df_features.columns)}")
print(f"   - New features added: {len(df_features.columns) - len(df_clean.columns)}")
print("\n6. NEXT STEPS:")
print("   - Data is ready for exploratory analysis")
print("   - Features are prepared for model training")
print("   - Proceed to notebook 02_eda.ipynb for detailed analysis") 